In [2]:
# Import key librarys

import pandas as pd
import numpy as np

import plotly.express as px

import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls

In [24]:
# Define level
level = 'Advanced Higher'
level_label = level.replace('_', ' ')

# Import data
years = ['2018', '2017', '2016', '2015', '2014']

fields = ['Title','National Rating']

li = []

for year in years:
    
    df = pd.read_excel('./ratings/National Ratings ' + year + '.xlsx', usecols = fields,
                        sheet_name = level, skiprows = 5)
    
    df['Year'] = year
    
    li.append(df)

In [25]:
years = ['2013', '2012','2011','2010','2009','2008','2007','2006']

for year in years:
    
    fields = ['Title','National Rating']
    rows = 5
    
    # Deal with different format of SQA spreadsheets over the years
    
    if level == 'National 5' or level == 'Standard Grade' or level == 'SG':
    
        if year >= '2012':
            level = 'Standard Grade'
        else:
            level = 'SG'
            
        if year <= '2010':
            rows = 6
        
        if year <= '2008':
            rows = 5
                
    if level == 'Advanced Higher' and year <= '2008':
        level = 'Adv Higher'
        
    if year == '2006':
        rows = 4

    df = pd.read_excel('./ratings/National Ratings ' + year + '.xls', usecols = fields,
                        sheet_name = level, skiprows = rows)
    
    df['Year'] = year
    
    li.append(df)

In [26]:
# concat list into long df

ldf = pd.concat(li, axis = 0, ignore_index = True)

In [27]:
ldf.tail()

,National Rating,Title,Year
478,0.199863,"Religious, Moral and Philosophical Studies",2006
479,NaN,Russian,2006
480,NaN,Sociology,2006
481,0.046842,Spanish,2006
482,0.162924,Technological Studies,2006


In [28]:
# sort colum titles
ldf = ldf.rename({'Title': 'Subject'}, axis = 1)

# remove rows with NaN
ldf.dropna(inplace = True)

# round to 2 decimal places
ldf = ldf.round(2)

In [29]:
# Sort out subjects names in 2016 and 2017 datasets

def tidy_up(df):

    # Update names
    di = {'Home Economics: Fashion and Textile Technology': 'Fashion and Textile Technology',
            'Home Economics: Health and Food Technology': 'Health and Food Technology',
            'Home Economics: Lifestyle and Consumer Technology': 'Lifestyle and Consumer Technology',
            'Administration': 'Administration and IT',
            'Computing': 'Computing Science',
            'Dance Practice': 'Dance',
            'Managing Environmental Resources': 'Environmental Science',
            'Media': 'Media Studies',
            'Product Design': 'Design and Manufacture',
            'Art and Design: Design': 'Art and Design (Design)',
            'Art and Design: Expressive': 'Art and Design (Expressive)',
            'Art and Design Enquiry: Design': 'Art and Design (Design)',
            'Art and Design Enquiry: Expressive': 'Art and Design (Expressive)',
            'Gaidhlig': 'Gàidhlig',
            'Accounting': 'Accounting & Finance',
            'Art & Design': 'Art and Design',
            'Religious Studies': 'Religious, Moral and Philosophical Studies',
            'Computing Studies': 'Computing Science',
            'Home Economics': 'Health and Food Technology',
            'Photography for the Media': 'Photography'}
    
    df = df.replace({"Subject": di})
    
    # Remove revised science courses
    values = ['Biology (Revised)', 'Chemistry (Revised)', 'Physics (Revised)', 'Human Biology (Revised)']
    
    df = df.loc[~df['Subject'].isin(values)]
    
    # Remove other courses that don't seem useful
    values = ['Religious, Moral and Philosophical Studies (New)', 'Selling Scheduled Air Travel',
                 'Building Construction', 'Construction']
    
    df = df.loc[~df['Subject'].isin(values)]
    
    return df

ldf = tidy_up(ldf)

In [30]:
# Plot graph

# Create a list of subjects in alphabetical order
subjects = sorted(ldf['Subject'].unique())
years = sorted(ldf['Year'].unique())

fig = px.line(ldf, x = "Year", y = "National Rating", 
                color = "Subject",
                width = 1000,
                height = 800,
                category_orders = {'Subject' : subjects, 'Year' : years},
               # color_discrete_sequence=px.colors.qualitative.Light24,
                title = 'National ratings per subject from 2006 - 2018 at ' + level_label #+ ' and Standard Grade'
                )

fig.update_traces(mode = "markers+lines")

fig.update_yaxes(matches = None, rangemode = "tozero")

filename = 'ratings_' + level + '.html'

fig.write_html('./graphs/' + filename)

fig.show()

In [3]:
# Setup Chart Studio
username = '###' # your username
api_key = '###' # your api key - go to profile > settings > regenerate key
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

# Upload to Chart Studio
#py.plot(fig, filename = filename, auto_open=True)